# imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
import spacy

In [3]:
import en_core_web_sm as english

In [4]:
from gensim.utils import any2unicode

In [5]:
import time

In [6]:
import cPickle

In [7]:
def dump(data, filename):
    with open(filename + ".pickle", "wb") as output_file:
        cPickle.dump(data, output_file)

In [8]:
def load(filename):
    with open(filename + ".pickle", "rb") as input_file:
        e = cPickle.load(input_file)
        return e

# Data Loading

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [12]:
train.shape

(159571, 8)

In [13]:
test.shape

(153164, 2)

# exploration

In [14]:
train.loc[train.insult==1]['comment_text']

6              COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
42        You are gay or antisemmitian? \n\nArchangel WH...
43                 FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!
55        Stupid peace of shit stop deleting my stuff as...
56        =Tony Sidaway is obviously a fistfuckee. He lo...
65        All of my edits are good.  Cunts like you who ...
86        Would you both shut up, you don't run wikipedi...
105              A pair of jew-hating weiner nazi schmucks.
168       You should be fired, you're a moronic wimp who...
176       I think that your a Fagget get a oife and burn...
179       REPLY ABOVE:\nThat was me, loser. The UN defin...
181       you are a stupid fuck \n\nand your mother's cu...
211                   Fuck you, block me, you faggot pussy!
218       Kill all niggers. \n\nI have hard, that others...
231       Burn Deck \n\nIf that'd guy's burn deck is lik...
238       u r a tw@ fuck off u gay boy.U r smelly.Fuck u...
295       this user is such a worthless 

# checks

In [15]:
# check for nulls
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [16]:
test.isnull().sum()

id              0
comment_text    0
dtype: int64

In [17]:
# check for imbalance classes
print 'toxic: ', train.toxic.value_counts()
print 'severe_toxic: ', train.severe_toxic.value_counts()
print 'obscene: ', train.obscene.value_counts()
print 'threat: ', train.threat.value_counts()
print 'insult: ', train.insult.value_counts()
print 'identity hate: ', train.identity_hate.value_counts()

toxic:  0    144277
1     15294
Name: toxic, dtype: int64
severe_toxic:  0    157976
1      1595
Name: severe_toxic, dtype: int64
obscene:  0    151122
1      8449
Name: obscene, dtype: int64
threat:  0    159093
1       478
Name: threat, dtype: int64
insult:  0    151694
1      7877
Name: insult, dtype: int64
identity hate:  0    158166
1      1405
Name: identity_hate, dtype: int64


# Training

In [18]:
test_vectors_ndarray = load('test_vectors_ndarray')

In [19]:
train_vectors_ndarray = load('train_vectors_ndarray')

In [18]:
x_train = train[['id', 'comment_text']]
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [19]:
x_train.head()

,id,comment_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember..."


In [20]:
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [21]:
y_train.as_matrix().shape

(159571, 6)

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [23]:
import keras
print keras.__version__

1.2.2


## Baseline - 6L MLP with Adam: 0.9604 public LB

In [146]:
mlp = Sequential()
mlp.add(Dense(400, input_shape=(300,), activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(200, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(100, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(50, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(6, activation='sigmoid'))

mlp.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [147]:
file_path="weights_base_6L_adam.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early

In [148]:
mlp.fit(train_vectors_ndarray, y_train.as_matrix(), nb_epoch=100, batch_size=256, verbose=1, validation_split=0.2,
       callbacks = callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/100
127656/127656 [==============================] - 3s - loss: 0.1213 - acc: 0.9530 - val_loss: 0.0641 - val_acc: 0.9769
Epoch 2/100
127656/127656 [==============================] - 2s - loss: 0.0722 - acc: 0.9766 - val_loss: 0.0625 - val_acc: 0.9786
Epoch 3/100
127656/127656 [==============================] - 2s - loss: 0.0686 - acc: 0.9775 - val_loss: 0.0620 - val_acc: 0.9784
Epoch 4/100
127656/127656 [==============================] - 2s - loss: 0.0666 - acc: 0.9779 - val_loss: 0.0603 - val_acc: 0.9790
Epoch 5/100
127656/127656 [==============================] - 2s - loss: 0.0656 - acc: 0.9782 - val_loss: 0.0626 - val_acc: 0.9784
Epoch 6/100
127656/127656 [==============================] - 2s - loss: 0.0645 - acc: 0.9784 - val_loss: 0.0589 - val_acc: 0.9796
Epoch 7/100
127656/127656 [==============================] - 2s - loss: 0.0641 - acc: 0.9784 - val_loss: 0.0586 - val_acc: 0.9797
Epoch 8/100
127656/127656 [============

# GRU: 0.9828 public LB, 622 Rank

## Best GRU:

In [131]:
gru.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_7 (Embedding)          (None, 250, 200)      20000000    embedding_input_7[0][0]          
____________________________________________________________________________________________________
bidirectional_7 (Bidirectional)  (None, 250, 256)      252672      embedding_7[0][0]                
____________________________________________________________________________________________________
globalmaxpooling1d_1 (GlobalMaxP (None, 256)           0           bidirectional_7[0][0]            
____________________________________________________________________________________________________
dropout_18 (Dropout)             (None, 256)           0           globalmaxpooling1d_1[0][0]       
___________________________________________________________________________________________

parameters:

dropout_U = 0.1

dropout_W = 0.1

dropout_dense1 = 0.1

max_features = 100000

max_len = 250

vector_len = 200

batch size= 512

validation: 0.1

Epoch 1/100
143360/143613 [============================>.] - ETA: 0s - loss: 0.0345 - acc: 0.9863Epoch 00000: val_loss improved from 0.04145 to 0.04144, saving model to weights_gru_128BiGru_dropouts0pt1_dense128_BS512_adam.best.hdf5

 ROC-AUC - epoch: 1 - score: 0.988243 

143613/143613 [==============================] - 261s - loss: 0.0345 - acc: 0.9863 - val_loss: 0.0414 - val_acc: 0.9844

EXPERIMENTS:

1. Bi GRU with 256 Dense and 64 GRU with 0.1 dropout and 32 BS yielded 0.9814 accuracy, max_len=250

2. Bi GRU with 128 Dense and 64 GRU with 0.2 dropout and 256 BS yielded 0.9815 accuracy, max_len=250

3. Bi GRU with 128 Dense and 64 GRU with 0.1 dropout and 512 BS yielded 0.9828 accuracy, max_len=250. Increasing  batch size decreases training time, does not reduce score. 0.1 dropout is good, as well as 128 dense. Higher values for both reduced score. 

4. Batch Normalization after Dropout in Dense did not help, reduced score

5. increasing dropouts to 0.5 was not at all good

6. increasing dropout to 0.2 with BS 512 and CNN+GRU and LR exp decay produced local AUC of 0.9870 but 0.9810 on public LB. - keep it for final submission

7. BS32 with CNN+GRU reduced the public LB to 0.9732 after 6 epochs

8. CNN after GRU is not good. increasing GRU output decreased val_loss and lead to 0.9831 on public LB. The network was further trained using 0.0001 LR with exp decay. Maybe try reducing the LR further, or train complete model using LR with exp decay.

9. Bi-GRU 128 with dense 128 trained with exp_decay achieved good CV ROC_AUC, but scored .9831 on public LB.

In [24]:
from keras.layers import Embedding

In [25]:
from keras.preprocessing.text import Tokenizer

In [26]:
from keras.preprocessing.sequence import pad_sequences

In [27]:
from keras.layers import GRU, Dense

In [28]:
from keras.layers import GlobalMaxPooling1D

In [29]:
from keras.layers import Bidirectional

In [30]:
from keras.layers import BatchNormalization

In [31]:
import numpy as np

In [32]:
from keras.layers import Convolution1D, MaxPooling1D, Flatten, Reshape

In [33]:
from keras.callbacks import Callback

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
from sklearn.metrics import roc_auc_score

In [36]:
from keras import backend as K

In [37]:
from keras.callbacks import TensorBoard

In [38]:
import time

In [39]:
EMBEDDING_FILE='/media/sarthak/HDD/data_science/Kaggle/jigsaw_toxic/glove.twitter.27B.200d.txt'

In [40]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [41]:
embeddings_index['in'] # random word vector

array([  5.75499982e-02,  -7.29900002e-02,   5.42400002e-01,
         7.70530030e-02,   1.68520004e-01,   4.20570001e-02,
        -6.92030013e-01,  -5.26000023e-01,  -3.75719994e-01,
        -7.64179975e-02,   2.78530002e-01,   5.62749982e-01,
        -1.43410003e+00,   4.01129991e-01,  -3.78769994e-01,
        -2.71349996e-01,   2.49200001e-01,  -5.66839993e-01,
        -3.79669994e-01,  -3.82070005e-01,  -1.13830000e-01,
        -2.23000005e-01,   4.32830006e-02,   2.27819994e-01,
         3.83630008e-01,   9.02469993e-01,   1.81920007e-01,
        -7.07949977e-03,  -2.51349986e-01,  -4.29110005e-02,
        -5.85730016e-01,  -8.31459984e-02,   2.34549999e-01,
        -3.40930015e-01,   4.00620013e-01,  -1.95230007e-01,
         1.27010003e-01,   3.14830005e-01,   2.87770003e-01,
         2.64070004e-01,  -3.61030012e-01,  -5.60890019e-01,
        -2.80950010e-01,   5.09440005e-01,   2.81580001e-01,
        -6.95409998e-02,   2.37440005e-01,  -6.76280037e-02,
        -2.67230004e-01,

In [42]:
list_sentences_train = train.comment_text
list_sentences_test = test.comment_text

In [43]:
max_unique_words = 100000
max_len = 250
tokenizer = Tokenizer(nb_words=max_unique_words)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [44]:
embed_size = 200

In [45]:
tokenizer.word_index

{"dool's": 44257,
 '\xca\x8a': 96087,
 'sowell': 65677,
 'tsukino': 96089,
 '\xca\x84': 96090,
 "agencies'": 202376,
 'woods': 8638,
 'spiders': 27398,
 'gavan': 65678,
 'dekolb': 47304,
 'ultimatley': 96092,
 'woody': 15188,
 'trawling': 22036,
 'comically': 44258,
 '027597675': 96093,
 '\xe2\x80\x9emerkspruch\xe2\x80\x9d': 159400,
 'regularize': 96094,
 'alwiqi': 96095,
 'eastasia': 65679,
 'sprague': 65680,
 'caney': 65688,
 'jairam': 96096,
 'acurately': 96097,
 'supasoldier': 96098,
 'falseinformation': 172137,
 '\xe2\x80\x98west\xe2\x80\x99': 96099,
 'rickman': 96100,
 'foundation\xe2\x80\x99s': 96101,
 'riconferma': 52273,
 'dra\xc5\xbea': 96102,
 'riconferme': 65698,
 '\xd0\xa2\xd0\xb5\xd0\xbe\xd0\xb4\xd0\xbe\xd1\x80': 175594,
 'naturopathic': 20228,
 "wood'": 96104,
 'sidestrand': 44259,
 'pigment': 38869,
 'occops': 96105,
 '41f1972e': 204714,
 'tijfo098': 96106,
 'bringing': 2801,
 'raviah': 96107,
 'jrpg': 52274,
 'tcby': 96108,
 'wooded': 52275,
 'distributerd': 75657,
 'k

In [46]:
X_t = pad_sequences(list_tokenized_train, maxlen=max_len)
X_te = pad_sequences(list_tokenized_test, maxlen=max_len)

In [47]:
len(X_t)

159571

In [48]:
len(X_te)

153164

In [49]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.0090650525, 0.44281197)

In [50]:
word_index = tokenizer.word_index
nb_words = min(max_unique_words, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_unique_words: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [51]:
embedding_matrix.shape

(100000, 200)

In [52]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0, batch_size=512)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [53]:
[X_tra, X_val, y_tra, y_val] = train_test_split(X_t, y_train.values, train_size=0.90, random_state=233)

/media/sarthak/HDD/Anaconda/envs/riskmethods_27/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [54]:
print(X_tra.shape)
print(X_val.shape)

(143613, 250)
(15958, 250)


In [82]:
batch_size = 256
epochs = 100

In [56]:
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(X_tra)/batch_size) * epochs

In [78]:
gru = Sequential()
gru.add(Embedding(input_dim=max_unique_words, output_dim=embed_size, weights=[embedding_matrix], trainable=False,
                 input_length=max_len)),
gru.add(Bidirectional(GRU(256, return_sequences=True,name='gru_1', dropout_U=0.1, dropout_W=0.1)))
gru.add(GlobalMaxPooling1D())
gru.add(Dropout(0.3))
gru.add(Dense(128, activation="relu"))
gru.add(Dropout(0.1))
#gru.add(BatchNormalization())
gru.add(Dense(6, activation="sigmoid"))
#gru.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu', 
                      #input_length=(None, None, 128)))
#gru.add(MaxPooling1D(pool_length=2, stride=2))
#gru.add(Flatten())
#gru.add(Dropout(0.5))
#gru.add(Dense(128, activation="relu"))
#gru.add(Dropout(0.1))
#gru.add(Dense(6, activation="sigmoid"))
gru.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
tensorboard = TensorBoard(log_dir='./logs')


In [79]:
lr_init, lr_fin = 0.001, 0.0005
lr_decay = exp_decay(lr_init, lr_fin, steps)
K.set_value(gru.optimizer.lr, lr_init)
K.set_value(gru.optimizer.decay, lr_decay)

In [65]:
# current model architecture
gru.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 250, 200)      20000000    embedding_input_2[0][0]          
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 250, 256)      252672      embedding_2[0][0]                
____________________________________________________________________________________________________
globalmaxpooling1d_2 (GlobalMaxP (None, 256)           0           bidirectional_2[0][0]            
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 256)           0           globalmaxpooling1d_2[0][0]       
___________________________________________________________________________________________

In [80]:
file_path="weights_gru_256BiGru_dropouts0pt1_dense128_BS512_expdecay_adam.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early, RocAuc] #early

In [83]:
gru.fit(X_tra,y_tra, batch_size=batch_size, nb_epoch=epochs, verbose=1, validation_data=(X_val, y_val), 
        callbacks = callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/100
143360/143613 [============================>.] - ETA: 0s - loss: 0.0658 - acc: 0.9775Epoch 00000: val_loss improved from inf to 0.04718, saving model to weights_gru_256BiGru_dropouts0pt1_dense128_BS512_expdecay_adam.best.hdf5

 ROC-AUC - epoch: 1 - score: 0.979955 

143613/143613 [==============================] - 427s - loss: 0.0657 - acc: 0.9775 - val_loss: 0.0472 - val_acc: 0.9827
Epoch 2/100
143360/143613 [============================>.] - ETA: 0s - loss: 0.0462 - acc: 0.9825Epoch 00001: val_loss improved from 0.04718 to 0.04429, saving model to weights_gru_256BiGru_dropouts0pt1_dense128_BS512_expdecay_adam.best.hdf5

 ROC-AUC - epoch: 2 - score: 0.985594 

143613/143613 [==============================] - 427s - loss: 0.0461 - acc: 0.9825 - val_loss: 0.0443 - val_acc: 0.9833
Epoch 3/100
143360/143613 [============================>.] - ETA: 0s - loss: 0.0426 - acc: 0.9835Epoch 00002: val_loss improved from 0.04429 to 0.0

KeyboardInterrupt: 

# prediction and submission

In [84]:
gru.load_weights(file_path)
y_test = gru.predict(X_te, verbose=1, batch_size=256)

153164/153164 [==============================] - 153s   


In [85]:
y_test

array([[  9.98168468e-01,   3.63719732e-01,   9.73769248e-01,
          5.31068370e-02,   8.98709536e-01,   5.89170694e-01],
       [  3.09806666e-04,   9.59095061e-08,   1.26369432e-05,
          6.25580014e-07,   2.91981560e-05,   3.43879009e-07],
       [  1.11715388e-04,   2.69454148e-07,   9.54684856e-06,
          1.35619655e-06,   9.94454422e-06,   1.07646292e-06],
       ..., 
       [  6.85018022e-05,   1.63396638e-08,   7.76599609e-06,
          9.47088168e-08,   4.81047482e-06,   2.03495404e-07],
       [  3.96419870e-04,   2.26479429e-07,   1.74015677e-05,
          1.64334813e-06,   1.89090206e-05,   4.82305222e-05],
       [  9.80085313e-01,   1.19893076e-02,   7.55342305e-01,
          9.66226216e-04,   5.15134931e-01,   2.17875186e-03]], dtype=float32)

In [86]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [87]:
sample_submission = pd.read_csv("sample_submission.csv")

In [88]:
sample_submission[list_classes] = y_test

In [89]:
sample_submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998168,3.637197e-01,0.973769,5.310684e-02,0.898710,5.891707e-01
1,0000247867823ef7,0.000310,9.590951e-08,0.000013,6.255800e-07,0.000029,3.438790e-07
2,00013b17ad220c46,0.000112,2.694541e-07,0.000010,1.356197e-06,0.000010,1.076463e-06
3,00017563c3f7919a,0.000346,1.863934e-07,0.000056,2.157662e-06,0.000057,6.031547e-07
4,00017695ad8997eb,0.002477,1.493587e-06,0.000089,2.831962e-05,0.000093,3.234728e-06
5,0001ea8717f6de06,0.000174,1.122055e-07,0.000021,7.131162e-07,0.000021,4.506473e-07
6,00024115d4cbde0f,0.001091,1.357754e-07,0.000049,1.255489e-06,0.000102,1.661784e-06
7,000247e83dcc1211,0.237570,8.948431e-05,0.011007,1.152769e-04,0.018218,1.560999e-04
8,00025358d4737918,0.029724,4.850974e-06,0.002690,4.336417e-05,0.003569,6.530054e-05
9,00026d1092fe71cc,0.000398,2.171362e-07,0.000037,2.698659e-06,0.000029,1.051280e-06


In [90]:
sample_submission.shape

(153164, 7)

In [91]:
sample_submission.to_csv("gru_13.csv", index=False)